### chatbot
- "번역:내용(한글)" -> 영문
- "날씨:지역" -> 지역날씨정보(한글)

- 네이버 파파고 : 한글 -> 영문, 영문 -> 한글
- 카카오 로컬 : 지역이름 -> 위도, 경도
- darksky API : 위도, 경도 -> 날씨정보(영문)

- 서울 -> 카카오 로컬(위도, 경도) -> darksky(영문 날씨) -> 네이버 파파고(한글 날씨) -> 슬랙 전송

- outgoing webhook : POST 방식

In [9]:
naver_id = "wzFa_hXu9Kjp"
naver_secret = "JpHIfF7"
kakao_key = "161f5ce3b5"
darksky_key = "1fbca7d24d235cdf"
webhook_url = "https://hooks.slack.com/services/T0L479/B01FSG/Uy7XFN"

In [10]:
!mkdir -p chatbot/libs
!touch chatbot/chatbot.py
!touch chatbot/libs/naver.py
!touch chatbot/libs/weather.py
!touch chatbot/libs/slack.py

In [11]:
!tree chatbot/

chatbot/
├── chatbot.py
└── libs
    ├── naver.py
    ├── slack.py
    └── weather.py

1 directory, 4 files


In [12]:
# 1. 네이버 번역 모듈

In [13]:
%%writefile chatbot/libs/naver.py
import requests, json

def translate(naver_id, naver_secret, text, source="ko", target="en"):
    url = "https://openapi.naver.com/v1/papago/n2mt"
    params = {"source": source, "target": target, "text": text}
    headers = {
        "X-Naver-Client-Id": naver_id,
        "X-Naver-Client-Secret": naver_secret,
        "Content-Type": "application/json",
    }
    response = requests.post(url, json.dumps(params), headers=headers)
    return response.json()["message"]["result"]["translatedText"]

Overwriting chatbot/libs/naver.py


In [14]:
import chatbot.libs.naver as naver

In [15]:
text = "파이썬은 재미있습니다."
msg = naver.translate(naver_id, naver_secret, text)
msg

'Python is fun.'

In [16]:
# 2. 날씨 정보 모듈 (카카오)

In [44]:
%%writefile chatbot/libs/forecast.py
import requests
# import chatbot.libs.naver as naver # 테스트 할때 사용
import libs.naver as naver # flask 에서 실행할때 사용

def kakao_local(kakao_key, addr):
    url = "https://dapi.kakao.com//v2/local/search/address.json?query={}".format(addr)
    headers = {"Authorization": "KakaoAK {}".format(kakao_key)}
    response = requests.get(url, headers=headers)
    datas = response.json()["documents"][0]
    return datas["y"], datas["x"]

def weather(lat, lng, darksky_key):
    url = "https://api.darksky.net/forecast/{}/{},{}".format(darksky_key, lat, lng)
    response = requests.get(url)
    return response.json()["daily"]["summary"]

# 지역 > 위도,경도 > 영문 날씨 > 한글 날씨
def run(naver_id, naver_secret, kakao_key, darksky_key, addr):
    lat, lng = kakao_local(kakao_key, addr)
    en_msg = weather(lat, lng, darksky_key)
    ko_msg = naver.translate(naver_id, naver_secret, en_msg, "en", "ko")
    return "{}의 날씨는 {}".format(addr, ko_msg)

Overwriting chatbot/libs/forecast.py


In [33]:
addr = "부산"
lat, lng = kakao_local(kakao_key, addr)
lat, lng

NameError: name 'kakao_local' is not defined

In [19]:
en_msg = weather(lat, lng, darksky_key)
en_msg

NameError: name 'weather' is not defined

In [20]:
run(naver_id, naver_secret, kakao_key, darksky_key, addr)

Exception: File `'(naver_id,.py'` not found.

In [21]:
# 3. 슬랙 전송 모듈

In [34]:
import chatbot.libs.forecast as forecast

addr = "부산"
msg = forecast.run(naver_id, naver_secret, kakao_key, darksky_key, addr)
msg


'부산의 날씨는 오늘 비가 올 것 같다.'

In [35]:
%%writefile chatbot/libs/slack.py
import requests, json

def send_msg(webhook_url, mag, channel="#dss", username="슬랙봇"):
    payload = {"channel": channel, "username": username, "text": mag}
    requests.post(webhook_url, data=json.dumps(payload))

Overwriting chatbot/libs/slack.py


In [36]:
import chatbot.libs.slack as slack
slack.send_msg(webhook_url, msg)

In [37]:
# 4. chatbot.py 작성

In [38]:
%%writefile chatbot/chatbot.py
from flask import *

import libs.naver as naver
import libs.forecast as forecast
import libs.slack as slack

app = Flask(__name__)

naver_id = "iz9ZHA"
naver_secret = "l"
kakao_key = "574f770c"
darksky_key = "1fbca7d24"
webhook_url = "https://hooks.slack.com/services/TNR/CKY/f2eiwIb"

@app.route("/")
def index():
    return "server is running!!!"

@app.route("/bot", methods=["POST"])
def bot():
    
    username = request.form.get("user_name")
    token = request.form.get("token")
    text =  request.form.get("text")
    
    print(username, token, text)
    
    text = text.replace("[bot] ", "")
    
    if text.find(":") < 0:
        slack.send_msg(webhook_url, "(명령):(데이터) 포멧으로 입력해주세요.")
        return Response(), 200
    
    # [bot] 날씨:부산
    comm, data = text.split(":")[0], text.split(":")[1]
    
    # [bot] 번역:파이썬을 재미있습니다.
    if comm == "번역":
        msg = naver.translate(naver_id, naver_secret, data) # msg : python is fun!
        slack.send_msg(webhook_url, msg)
    elif comm == "날씨":
        msg = forecast.run(naver_id, naver_secret, kakao_key, darksky_key, data)
        slack.send_msg(webhook_url, msg)
    
    return Response(), 200


app.run(debug=True)

Overwriting chatbot/chatbot.py


In [39]:
# 5. 모델 파일 적용

In [40]:
!mkdir chatbot/models
!cp model.pkl chatbot/models

mkdir: cannot create directory ‘chatbot/models’: File exists


In [41]:
!tree chatbot/

chatbot/
├── chatbot.py
├── libs
│   ├── __pycache__
│   │   ├── forecast.cpython-36.pyc
│   │   ├── naver.cpython-36.pyc
│   │   └── slack.cpython-36.pyc
│   ├── forecast.py
│   ├── naver.py
│   ├── slack.py
│   └── weather.py
└── models
    └── model.pkl

3 directories, 9 files


In [43]:
%%writefile chatbot/chatbot.py
from flask import *

import pickle
import libs.naver as naver
import libs.forecast as forecast
import libs.slack as slack

app = Flask(__name__)

naver_id = "iz97KIA"
naver_secret = "8x"
kakao_key = "574f77753270c"
darksky_key = "1fbca7d24d65c5cdf"
webhook_url = "https://hooks.slack.com/services/TNKJR/B01FEQK4KY/f2eiNzKodwIb"

# load model file
with open("models/model.pkl", "rb") as f:
    model = pickle.load(f)


@app.route("/")
def index():
    return "server is running!!!"

@app.route("/bot", methods=["POST"])
def bot():
    
    username = request.form.get("user_name")
    token = request.form.get("token")
    text =  request.form.get("text")
    
    print(username, token, text)
    
    text = text.replace("[bot] ", "")
    
    if text.find(":") < 0:
        slack.send_msg(webhook_url, "(명령):(데이터) 포멧으로 입력해주세요.")
        return Response(), 200
    
    # [bot] 날씨:부산
    comm, data = text.split(":")[0], text.split(":")[1]
    
    # [bot] 번역:파이썬을 재미있습니다.
    if comm == "번역":
        msg = naver.translate(naver_id, naver_secret, data) # msg : python is fun!
        slack.send_msg(webhook_url, msg)
    elif comm == "날씨":
        msg = forecast.run(naver_id, naver_secret, kakao_key, darksky_key, data)
        slack.send_msg(webhook_url, msg)
    
    elif comm == "카테고리":
        datas = ['정치', '경제', '사회', '생활/문화', '세계', 'IT/과학']
        idx = model.predict([data])[0]-100
        slack.send_msg(webhook_url, msg)
    
    return Response(), 200


app.run(debug=True)

Overwriting chatbot/chatbot.py
